In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib widget

from toolz import *

import os
import pickle
import json
import pathlib

import glom
from lenses import lens

In [2]:
lmap = compose(list, map)

In [3]:
def list_of_str_to_list_of_json(lst):
    it = map(lambda x: x.replace('\xa0', ' '), lst)
    it = filter(lambda x: x != '', it)
    it = lmap(json.loads, it)
    return it

In [4]:
data_path = pathlib.Path('data')

In [5]:
instance_entry_lens_get = lens['Завтраки'][3].get()

## Loading dataset from files (as nested structure):

In [6]:
dict_json_by_course = {}
for path_course in pathlib.Path('data').joinpath('collected').iterdir():
    list_of_course_jsons = []
    for path_file in path_course.iterdir():
        with open(path_file, 'r', encoding='utf-8') as f:
            list_of_course_jsons += list_of_str_to_list_of_json(f.readlines())
    dict_json_by_course[str(path_course.name)] = list_of_course_jsons

In [7]:
dict_json_by_course.keys()

dict_keys(['Выпечка и десерты', 'Завтраки', 'Закуски', 'Напитки', 'Основные блюда', 'Паста и пицца', 'Салаты', 'Соусы и маринады', 'Супы'])

In [8]:
instance_entry_lens_get(dict_json_by_course)

{'url': 'https://eda.ru/recepty/zavtraki/zapekanka-iz-lapshi-vetchini-schinkenfleckerln-22024',
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'category': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'menu': 'Еда без ограничений',
 'description': 'Запеканка из макарон — вовсе не странность: ашкеназский кугель, русский лапшевик, итальянская лазанья, листы для которой тоже относятся к пасте, так что австрийский шинкенфлекерльн в хорошей компании. Приготовить это блюдо проще, чем выговорить его название: сочетание лапши, ветчины и сыра, конечно, сплошные калории, зато ученые все больше склоняются к тому, что жиры и углеводы действительно могут помочь нам справиться со стрессом. Или можете заменить жирные сорта ветчины, к примеру, вареной индейкой, сразу сбросив часть груза калорий. К тому же готовится такая запеканка не дольше, чем яичница, вполне укладывается даже в утренний тайминг и позволяет пустить в дело замыленный кусочек сыра или остатки вчерашних макарон. Если бу

## Processing:

Now we're ready to process this data to create dataset for our needs

In [9]:
# lens that yields seperate dicts from the data
# (like `instance_entry_lens_get(dict_json_by_course)` above)
data_processing_lens = lens.Values().Each()

### First of all, renaming `category` to `course`:

In [10]:
dict_json_by_course_renamed = data_processing_lens.Keys().modify(
    lambda field_name: 'course' if field_name == 'category' else field_name,
)(dict_json_by_course)

example_instance = instance_entry_lens_get(dict_json_by_course_renamed)
example_instance

{'url': 'https://eda.ru/recepty/zavtraki/zapekanka-iz-lapshi-vetchini-schinkenfleckerln-22024',
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'course': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'menu': 'Еда без ограничений',
 'description': 'Запеканка из макарон — вовсе не странность: ашкеназский кугель, русский лапшевик, итальянская лазанья, листы для которой тоже относятся к пасте, так что австрийский шинкенфлекерльн в хорошей компании. Приготовить это блюдо проще, чем выговорить его название: сочетание лапши, ветчины и сыра, конечно, сплошные калории, зато ученые все больше склоняются к тому, что жиры и углеводы действительно могут помочь нам справиться со стрессом. Или можете заменить жирные сорта ветчины, к примеру, вареной индейкой, сразу сбросив часть груза калорий. К тому же готовится такая запеканка не дольше, чем яичница, вполне укладывается даже в утренний тайминг и позволяет пустить в дело замыленный кусочек сыра или остатки вчерашних макарон. Если буде

### Get rid of redundant data and generate `id`s:

In [14]:
id_from_url = lambda url: int(url.split('-')[-1])
def process_instance(instance):
    return glom.glom(instance, {
        'id': glom.Invoke(id_from_url).specs('url'),
        'title': glom.Invoke(str.strip).specs('title'),
        'course': 'course',
        'cuisine': 'cuisine',
        'ingredients': ('ingredients', glom.Iter().map(lambda x: x[0].strip()).all())
    })
process_instance(example_instance)

{'id': 22024,
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'course': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'ingredients': ['Ветчина',
  'Широкая лапша',
  'Репчатый лук',
  'Тертый сыр пармезан',
  'Соль',
  'Свежемолотый черный перец',
  'Оливковое масло']}

In [15]:
dict_json_by_course_processed = data_processing_lens.modify(process_instance)(dict_json_by_course_renamed)
instance_entry_lens_get(dict_json_by_course_processed)

{'id': 22024,
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'course': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'ingredients': ['Ветчина',
  'Широкая лапша',
  'Репчатый лук',
  'Тертый сыр пармезан',
  'Соль',
  'Свежемолотый черный перец',
  'Оливковое масло']}

### We're ready to flatten our data ang turn it into dataframe:

In [16]:
all_instances_flattened = data_processing_lens.collect()(dict_json_by_course_processed)
all_instances_flattened[0]

{'id': 28195,
 'title': 'Банановое мороженое с корицей',
 'course': 'Выпечка и десерты',
 'cuisine': 'Карибская кухня',
 'ingredients': ['Бананы',
  'Лимонный сок',
  'Молотая корица',
  'Ванильное мороженое']}

In [18]:
df = pd.DataFrame(all_instances_flattened)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[Бананы, Лимонный сок, Молотая корица, Ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[Сахар, Кокосовое молоко, Корень имбиря, Анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[Клубника, Манго, Сахар, Тертая цедра лайма, Т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[Манго, Темный ром, Сок лайма, Сахар, Тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[Клубника, Сахарная пудра, Сок лайма, Текила, ..."
...,...,...,...,...,...
42196,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[Белая фасоль, Овощной бульон, Свекла, Картофе..."
42197,80446,Суп энгамат,Супы,Шведская кухня,"[Цветная капуста, Морковь, Картофель, Лук-поре..."
42198,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[Горох, Репчатый лук, Гвоздика, Свиная рулька,..."
42199,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[Оливковое масло, Репчатый лук, Овощной бульон..."


In [23]:
it = df.groupby('id').count()
it[it['title'] > 2]

,title,course,cuisine,ingredients
id,,,,
40356,3,3,3,3
41279,3,3,3,3


In [ ]:
# with open('foodie_favorites_dataset/first_reviews.json', 'r') as f:
#     rewiews_lines = f.readlines() #json.load(f)

# rewiews_first = json.loads(rewiews_lines[1])

# with open('foodie_favorites_dataset/restaurants.json', 'r') as f:
#     for _ in range(1):
#         f.readline()
#     restaurant_nth = json.loads(f.readline())

# for i in range(len(rewiews_first['reviews'])):
#     print(rewiews_first['reviews'][i]['comment'])
#     print('\n' + '-'*100 + '\n')

list_vypechka = []
for file in data_path.joinpath('vypechka-deserty').iterdir():
    list_vypechka.append(file.read_text(encoding='utf-8'))

vypechka_path_to_add =  data_path.joinpath('data_file_vypechka_evropeyskaya.json')
with open(vypechka_path_to_add, 'r', encoding='utf-8') as f:
    list_vypechka += f.readlines()

vypechka_path_to_add =  data_path.joinpath('data_file.json')
with open(vypechka_path_to_add, 'r', encoding='utf-8') as f:
    list_vypechka += f.readlines()

for path in ['data_file_osnovnye_blyda.json',
             'data_file_Завтраки_Напитки_Паста_и_пицца_Соусы_и_маринады_Супы.json',
             'data_file_Закуски_Салаты.json']:
    path_obj = data_path.joinpath(path)
    with open(path_obj, 'r', encoding='utf-8') as f:
        path_list = f.readlines()
    json_form = json.dumps(list_of_str_to_list_of_json(list_vypechka), indent=2, ensure_ascii=False)
    path_obj.write_text(json_form)

json_form = json.dumps(list_of_str_to_list_of_json(list_vypechka), indent=2, ensure_ascii=False)
data_path.joinpath('vypechka-deserty.json').write_text(json_form)


In [34]:
res = {}
for elt in data_path.parent.joinpath('old').iterdir():
    if not elt.is_dir():
        res[str(elt)] = pd.json_normalize(json.loads(elt.read_text(encoding='utf-8')))

In [35]:
res_lst = list(res.values())
res_lst[0]

,title,category,cuisine,menu,description
0,Фаршированные белые грибы с сыром,Закуски,Русская кухня,Еда без ограничений,
1,"Рулеты из огурца, картофельного пюре и сельди",Закуски,Русская кухня,Еда без ограничений,
2,Глазированная репа,Закуски,Русская кухня,Еда без ограничений,
3,Баклажаны с грибами в сметанном соусе,Закуски,Русская кухня,Еда без ограничений,
4,Оладьи из малоглазого макруруса,Закуски,Русская кухня,Еда без ограничений,
...,...,...,...,...,...
7632,Мидии с пармезаном и белым вином,Салаты,Бельгийская кухня,Еда без ограничений,
7633,Салат с подкопченой семгой и хурмой,Салаты,Аргентинская кухня,Еда без ограничений,
7634,"Салат из мяса, овощей и моцареллы с соусом чим...",Салаты,Аргентинская кухня,Еда без ограничений,
7635,Биф-салат,Салаты,Аргентинская кухня,Еда без ограничений,


## Test stuff:

In [165]:
sample = {}
for i, key in enumerate(dict_json_by_course.keys()):
    sample[key] = []
    for j, lst_element in enumerate(dict_json_by_course_renamed[key]):
        sample[key].append(lst_element)
        sample[key][j]['ingredients'] = []
        sample[key][j]['instructions'] = []
        if j == 1: break
    if i == 2: break
sample

{'Выпечка и десерты': [{'url': 'https://eda.ru/recepty/vypechka-deserty/bananovoe-morozhenoe-s-koricej-28195',
   'title': 'Банановое мороженое с корицей',
   'course': 'Выпечка и десерты',
   'cuisine': 'Карибская кухня',
   'menu': 'Детское меню',
   'description': '',
   'ingredients': [],
   'instructions': [],
   'advice': ''},
  {'url': 'https://eda.ru/recepty/vypechka-deserty/ananasovo-kokosovij-sherbet-28141',
   'title': 'Ананасово-кокосовый шербет',
   'course': 'Выпечка и десерты',
   'cuisine': 'Карибская кухня',
   'menu': 'Еда без ограничений',
   'description': '',
   'ingredients': [],
   'instructions': [],
   'advice': ''}],
 'Завтраки': [{'url': 'https://eda.ru/recepty/zavtraki/yaichnica-po-abhazski-138276',
   'title': 'Яичница по-абхазски',
   'course': 'Завтраки',
   'cuisine': 'Абхазская кухня',
   'menu': 'Еда без ограничений',
   'description': 'Рецепт взят из книги «Абхазская кухня» издательства «Эксмо».',
   'ingredients': [],
   'instructions': [],
   'advic